# Assignment: Develop a Multi-Agent Financial Analyst with AutoGen Planner



### Objective:
This assignment challenges you to design and implement a **multi-agent financial analysis system using AutoGen**. The core of this system will be a **"Planner" agent** responsible for breaking down complex financial queries into actionable steps. Other agents (Researcher, Analyst, Reporter) will then execute these steps, leveraging external tools (like web search) to gather and synthesize financial information. This exercise demonstrates advanced multi-agent orchestration for specialized, data-intensive tasks.

---

### Instructions:
1.  **LLM Access**: You will need access to an LLM API. **OpenAI's models (e.g., GPT-4o, GPT-4, GPT-3.5-turbo)** are highly recommended due to their robust function calling capabilities, which AutoGen leverages extensively. You can also use local LLMs via `ollama` or `lmstudio` if configured correctly (see AutoGen docs).
2.  **External Tool Access (Search)**: You **must** use an internet search tool. **SerperDevTool** from `crewai_tools` (or a similar tool like `DuckDuckGoSearchRun` from `langchain_community.tools`) is recommended for its ease of integration. You will need an **API key for Serper (from serper.dev)** or configure other search tools.
3.  **Environment Setup**: Install the necessary Python libraries: `pip install pyautogen crewai_tools langchain_community`.
4.  **API Keys**: Securely handle your LLM API key and your Serper API key. It's best practice to load them from environment variables or `OAI_CONFIG_LIST`.
5.  **Jupyter Notebook**: All your code, outputs, observations, and analysis must be documented in this Jupyter Notebook.
6.  **Financial Analysis Scenario**: Define a specific financial analysis query (e.g., "Analyze the recent performance of NVIDIA (NVDA) stock and its future outlook, considering market trends and competitive landscape.").
7.  **Analysis**: Evaluate the planning process, agent collaboration, tool utilization, and the quality of the final report.

---

## Part 1: Setup and Configuration
Configure your LLM and external tools required for the financial analyst agents.

### Task 1.1: Install Libraries and Configure API Keys
Install `pyautogen`, `crewai_tools`, and `langchain_community`. Set up your LLM and Serper API keys.

In [ ]:
# Install necessary libraries (if not already installed)
# !pip install pyautogen crewai_tools langchain_community --quiet

import autogen
import os
from crewai_tools import SerperDevTool # Or from langchain_community.tools import DuckDuckGoSearchRun

# --- YOUR API KEYS HERE ---
# It's highly recommended to load your API keys from environment variables.
# On Linux/macOS: export OPENAI_API_KEY='your_openai_key' ; export SERPER_API_KEY='your_serper_key'
# On Windows (cmd): set OPENAI_API_KEY=your_openai_key & set SERPER_API_KEY=your_serper_key

# Replace with your actual keys or ensure environment variables are set
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "YOUR_OPENAI_API_KEY_HERE")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY", "YOUR_SERPER_API_KEY_HERE")

# Configure LLM (OpenAI model recommended)
llm_config = {
    "config_list": autogen.config_list_from_json(
        "OAI_CONFIG_LIST",
        filter_dict={
            "model": ["gpt-4o", "gpt-4", "gpt-3.5-turbo"], # Prioritize larger models
        },
    ),
    "temperature": 0.2, # Lower temperature for factual and analytical tasks
}

# Fallback if OAI_CONFIG_LIST not found
if not llm_config["config_list"] and os.environ.get("OPENAI_API_KEY"):
    print("Using OPENAI_API_KEY from environment variable as fallback.")
    llm_config["config_list"] = [
        {"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}
    ]
elif not llm_config["config_list"]:
    print("WARNING: No LLM configuration found. Please set OPENAI_API_KEY or create OAI_CONFIG_LIST.")


# Initialize the search tool
search_tool = SerperDevTool() # Or DuckDuckGoSearchRun()

print("AutoGen environment, LLM, and Search Tool configured!")

### Task 1.2: Define the Financial Analysis Query
Choose a specific financial analysis query that requires multiple steps, including research and analysis.

In [ ]:
financial_query = "Perform a comprehensive financial analysis of Tesla (TSLA) for the last two quarters. Include revenue, net income, cash flow, debt, and recent news impacting its stock performance. Provide a summary of its current financial health and a brief outlook."

print("Financial analysis query defined:", financial_query)

---

## Part 2: Define Agents (Roles and Tools)
Create the specialized AI agents that will form your financial analysis team. Each agent will have a distinct role and be equipped with necessary tools.

### Task 2.1: `UserProxyAgent` (The Initiator & Executor)
This agent initiates the conversation and can execute code (though likely not in this specific flow, it's good practice for AutoGen's capabilities). It also acts as the interface to the user and manages termination.

In [ ]:
# Define the termination condition for the UserProxyAgent
def is_termination_message(message):
    return "TERMINATE" in message.get("content", "").upper() or "FINISH" in message.get("content", "").upper()

user_proxy = autogen.UserProxyAgent(
    name="Admin",
    human_input_mode="NEVER", # Set to NEVER for full automation
    max_consecutive_auto_reply=15, # Allow enough turns for complex tasks
    is_termination_msg=is_termination_message,
    code_execution_config={
        "work_dir": "financial_analysis_workdir", # Directory for code execution
        "use_docker": False, # Set to True if you have Docker installed for isolated execution
    },
    system_message="A human administrator who manages the financial analysis team. Provide tasks and approve final reports. Conclude with 'TERMINATE' when satisfied."
)

print("UserProxyAgent 'Admin' created!")

### Task 2.2: `PlannerAgent` (The Strategist)
This agent will take the high-level financial query and break it down into a detailed, step-by-step plan for the other agents to follow. It's crucial for managing complex, multi-stage processes.

In [ ]:
planner = autogen.AssistantAgent(
    name="Planner",
    system_message="You are an expert financial analysis planner. Your role is to receive a financial query, break it down into a clear, sequential plan of action for research, analysis, and reporting. Delegate tasks to the Researcher, Analyst, and Reporter. Ensure all parts of the initial query are addressed. Output the plan and then instruct the next agent.",
    llm_config=llm_config,
)

print("PlannerAgent 'Planner' created!")

### Task 2.3: `ResearcherAgent` (The Data Gatherer)
This agent's primary responsibility is to gather relevant financial data and news using the search tool.

In [ ]:
researcher = autogen.AssistantAgent(
    name="Researcher",
    system_message=(
        "You are a highly skilled financial researcher. You are equipped with a web search tool. "
        "Your task is to use the search tool to find recent financial data, news articles, "
        "and relevant reports related to the query. Present the factual findings clearly, citing sources where possible. "
        "Do not analyze, just gather and present raw information. Once research is complete, return the findings to the Analyst."
    ),
    llm_config=llm_config,
)

# Register the search tool for the Researcher agent
user_proxy.register_for_execution(search_tool, caller=researcher)
researcher.register_for_tool_use(user_proxy, search_tool)

print("ResearcherAgent 'Researcher' created and search tool registered!")

### Task 2.4: `AnalystAgent` (The Interpreter)
This agent will take the raw research findings and perform the actual financial analysis, identifying trends, key metrics, and implications.

In [ ]:
analyst = autogen.AssistantAgent(
    name="Analyst",
    system_message=(
        "You are a highly analytical financial expert. Your role is to take the raw research findings "
        "from the Researcher and perform a thorough financial analysis. Identify key financial metrics "
        "(revenue, net income, cash flow, debt), trends, and significant news impacts. "
        "Synthesize these into concise insights. Present your analysis to the Reporter."
    ),
    llm_config=llm_config,
)

print("AnalystAgent 'Analyst' created!")

### Task 2.5: `ReporterAgent` (The Communicator)
This agent's task is to compile all the analysis into a well-structured, professional financial report.

In [ ]:
reporter = autogen.AssistantAgent(
    name="Reporter",
    system_message=(
        "You are a professional financial report writer. Your task is to take the analysis from the Analyst "
        "and compile it into a comprehensive, well-structured financial report. "
        "The report should have an introduction, sections for key financial data, trends, impact of news, "
        "a summary of financial health, and a brief outlook. Ensure clarity, conciseness, and professionalism. "
        "Present the final report to the Admin. Conclude your final message to Admin with 'TERMINATE'."
    ),
    llm_config=llm_config,
)

print("ReporterAgent 'Reporter' created!")

---

## Part 3: Define Communication Flow & Initiate Chat
Orchestrate the interaction between the agents using `GroupChat` and `GroupChatManager`.

### Task 3.1: Create `GroupChat` and `GroupChatManager`
Define the group of agents and the manager that will control their turn-taking and conversation flow.

In [ ]:
agents = [user_proxy, planner, researcher, analyst, reporter]

groupchat = autogen.GroupChat(
    agents=agents,
    messages=[],
    max_round=20, # Maximum number of turns in the conversation
    speaker_selection_method="auto", # AutoGen decides who speaks next
)

manager = autogen.GroupChatManager(
    groupchat=groupchat,
    llm_config=llm_config,
)

print("GroupChat and GroupChatManager created!")

### Task 3.2: Initiate the Multi-Agent Conversation
Start the chat by sending the `financial_query` to the `GroupChatManager` via the `user_proxy`.

In [ ]:
# Make sure the work directory exists
if not os.path.exists(user_proxy.code_execution_config["work_dir"]):
    os.makedirs(user_proxy.code_execution_config["work_dir"])

print("\n--- Initiating Financial Analysis Conversation ---")

chat_history = user_proxy.initiate_chat(
    manager, # The manager agent to start the group chat
    message=financial_query,
    clear_history=True, # Clear previous chat history
    silent=False, # Set to True to suppress console output during chat
)

print("\n--- Financial Analysis Conversation Ended ---")

---

## Part 4: Analysis and Reflection
Review the console output and the final report, then answer the following questions.

### Task 4.1: Review the Financial Analysis Report
* **Quality of Report**: Is the final financial analysis report comprehensive, accurate, and well-structured? Does it address all parts of the initial `financial_query`?
* **Key Findings**: What were the main financial insights provided about Tesla (TSLA) in the report?
* **Completeness**: Did the report include revenue, net income, cash flow, debt, and recent news as requested?


### Task 4.2: Role of the Planner Agent
* **Planning Effectiveness**: Observe the initial messages from the `Planner` agent. Did it effectively break down the complex financial query into logical, sequential steps?
* **Orchestration**: How did the `Planner` guide the conversation and delegate tasks to the other agents? Provide examples from the chat log.
* **Necessity**: Why is a `Planner` agent particularly beneficial in complex, multi-stage tasks like financial analysis, compared to a direct interaction between a `UserProxyAgent` and specialized agents?

### Task 4.3: Agent Collaboration and Tool Utilization
* **Collaboration Flow**: Describe the flow of information and collaboration between the `Planner`, `Researcher`, `Analyst`, and `Reporter` agents. How did each agent's output become the input for the next?
* **Search Tool Use**: Did the `Researcher` agent effectively use the `SerperDevTool`? Provide snippets from the log showing the tool calls and their results.
* **Efficiency**: Do you think this multi-agent setup was an efficient way to perform the analysis? Why or why not?

### Task 4.4: Limitations and Future Improvements
* **Current Limitations**: What were the limitations of this multi-agent setup (e.g., occasional inaccuracies, verbose output, potential for loops, lack of real-time data)?
* **Future Enhancements**: How could you improve this financial analyst system? Consider:
    * Adding more specialized agents (e.g., a 'Risk Assessor', 'Chart Generator').
    * Integrating more sophisticated tools (e.g., API access to financial data providers, a charting library).
    * Implementing a human-in-the-loop mechanism for critical decisions or clarifications.
    * Improving error handling and retry mechanisms.

---

### Submission:
* Ensure all code cells have been executed and their outputs are visible.
* All analysis and reflections are clearly written in markdown cells.
* Save your Jupyter Notebook as `[YourName]_AutoGen_Financial_Analyst_Assignment.ipynb`.